In [1]:
import numpy as np
import pandas as pd
# import scanpy.api as sc
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = (1,1,1,1)
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

In [2]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis', transparent=False, frameon=False, 
                             fontsize=20)  # low dpi (dots per inch) yields small inline figures

import matplotlib as mpl
# 2 lines below solved the facecolor problem.
# mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = (1,1,1,1)
sc.settings.autosave = True
sc.logging.print_versions()

version = '210711_merged_thymoma_MG21.22.23.03_Bcell'

# file_mat = 'F1314_200323_222536_summary/10425963-2_3DE_genematrix.csv'

results_file_Bcell_all = './scanpy/{}/merge_Bcell_all.h5ad'.format(version)
results_file_Bcell = './scanpy/{}/merge_Bcell.h5ad'.format(version)
# results_file_acell = './scanpy/{}/merge_abTcell.h5ad'.format(version)
results_file_Bcell_minor_cluster = './scanpy/{}/merge_Bcell_minor_cluster.h5ad'.format(version)
results_file_cg_Bcell_minor_cluster = './scanpy/{}/merge_cg_Bcell_minor_cluster.h5ad'.format(version)

results_file_master = './scanpy/210711_merged_thymoma_MG21.22.23.03/merge.h5ad'
results_raw_file_master = './scanpy/210711_merged_thymoma_MG21.22.23.03/merge.raw.h5ad'

# for cellphonedb
raw_matrix = './scanpy/{}/merge.raw.tsv'.format(version)

sc.settings.figdir = './scanpy/{}/graph'.format(version)
sc.settings.cachedir = './scanpy/{}/cache'.format(version)
# %config InlineBackend.figure_format = 'retina'

-----
anndata     0.7.6
scanpy      1.7.2
sinfo       0.3.1
-----
PIL                 8.1.2
anndata             0.7.6
backcall            0.2.0
cffi                1.14.5
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
decorator           4.4.2
get_version         2.1
google              NA
h5py                2.10.0
igraph              0.9.1
ipykernel           5.3.4
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.17.2
joblib              1.0.1
kiwisolver          1.3.1
legacy_api_wrap     1.2
leidenalg           0.8.4
llvmlite            0.34.0
matplotlib          3.4.1
mpl_toolkits        NA
natsort             7.1.1
numba               0.51.0
numexpr             2.7.3
numpy               1.20.2
packaging           20.9
pandas              1.2.4
parso               0.7.0
pexpect             4.8.0
pickleshare         0.7.5
pkg_resources       NA
prompt_toolkit      3.0.8
psutil              5.8.0
ptyproce

In [3]:
adata_raw = sc.read(results_raw_file_master)
adata = sc.read(results_file_master)

In [4]:
sc.pl.umap(adata, color='major_cluster', legend_loc='on data', title='', frameon=False, save='major_cluster_all')

<Figure size 320x320 with 1 Axes>

In [5]:
adata_cp = adata_raw[adata[adata.obs['major_cluster'] == 'B cell'].obs.index].copy()
    
adata = adata_cp.copy()
adata_cp = None

In [6]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
adata.raw = sc.pp.log1p(adata, copy=True)

normalizing by total count per cell
    finished (0:00:02): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


In [7]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [8]:
plt.figure(figsize=(8,3))
pd.Series(adata.obs['sample']).value_counts().plot(kind='bar')
plt.grid(None)
plt.title('Run')
# plt.savefig(str(sc.settings.figdir) + '/run.pdf', bbox_inches='tight')

Text(0.5, 1.0, 'Run')

<Figure size 640x240 with 1 Axes>

In [9]:
# adata = adata[adata.obs['sample'].isin(['MG03_TE', 'MG21_TE', 'MG22_TE'])]

In [10]:
adata = adata[:,(~ adata.var.index.str.startswith('IGKV')) & 
              (~ adata.var.index.str.startswith('IGLV')) &
             (~ adata.var.index.str.startswith('IGHV')) &
             (~ adata.var.index.str.startswith('IGLC'))]

filter_result = sc.pp.filter_genes_dispersion(
    adata.X, min_mean=0.0125, max_mean=2.5, min_disp=0.7)
sc.pl.filter_genes_dispersion(filter_result)
print([sum([i[0] for i in filter_result]),len(filter_result)])

extracting highly variable genes
    finished (0:00:00)


<Figure size 640x320 with 2 Axes>

[876, 36265]


In [11]:
adata = adata[:, filter_result.gene_subset]

sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata)
adata.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
sc.pl.pca_variance_ratio(adata, log=True)

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:375: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
computing PCA
    with n_comps=50
    finished (0:00:02)


<Figure size 320x320 with 1 Axes>

In [12]:
sc.external.pp.bbknn(adata, batch_key='sample')
sc.tl.umap(adata, spread=2)
# sc.tl.leiden(adata, resolution=1)

computing batch balanced neighbors
	finished: added to `.uns['neighbors']`
	`.obsp['distances']`, distances for each pair of neighbors
	`.obsp['connectivities']`, weighted adjacency matrix (0:00:03)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:14)


In [13]:
sc.pl.umap(adata, color=['n_counts', 'n_genes', 'pct_counts_mt'], save='qc')

<Figure size 1159.2x320 with 6 Axes>

In [14]:
sc.tl.leiden(adata, resolution=4)

running Leiden clustering
    finished: found 96 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:02)


In [15]:
sc.pl.umap(adata, color='leiden', legend_loc='on data', title='', frameon=False, save=False)

<Figure size 320x320 with 1 Axes>

In [16]:
sc.pl.dotplot(adata, var_names=['CD19', 'MS4A1', 'CD3E', 'CD4', 'CD8A', 'EPCAM', 'PECAM1', 
                         'IGHD', 'IGHM', 'IGHA1', 'IGHA2', 'IGHG1',
                         'IGHG2', 'IGHG3', 'IGHG4', 'IGHE'], groupby='leiden', swap_axes=True)

<Figure size 2961.6x528 with 4 Axes>

In [17]:
sc.tl.leiden(adata, resolution=0.7, restrict_to=('leiden',['28']), )

running Leiden clustering
    finished: found 102 clusters and added
    'leiden_R', the cluster labels (adata.obs, categorical) (0:00:00)


In [18]:
sc.pl.umap(adata, color='leiden_R', legend_loc='on data', title='', frameon=False, save=False)

<Figure size 320x320 with 1 Axes>

In [19]:
# SDC1 : CD138
sc.pl.dotplot(adata, var_names=['CD19', 'MS4A1', 'CD3E', 'CD4', 'CD8A', 'SDC1', 'EPCAM', 'PECAM1', 
                         'IGHD', 'IGHM', 'IGHA1', 'IGHA2', 'IGHG1',
                         'IGHG2', 'IGHG3', 'IGHG4', 'IGHE'], groupby='leiden_R', swap_axes=True)

<Figure size 3139.2x556 with 4 Axes>

In [20]:
# ITGAX : CD11C
# CD27 : memory B
sc.pl.umap(adata, color=['CD3E', 'CD4', 'CD8A', 'FOXP3', 'CTLA4', 'IL2RA', 'SATB1', 'EPCAM', 'PECAM1',
                         'MS4A1', 'CD27', 'BCL6', 'SCD',
                         'NKG7', 'TRDC', 'TRGC1', 'TRGC2', 'FCER1A', 'ITGAX', 'HLA-DQB1', 'MKI67', 'sample', 'site'],
          save='panel', ncols=5)

<Figure size 1932x1600 with 44 Axes>

In [21]:
sc.pl.umap(adata, color=['CD19', 'MS4A1', 'CD3E', 'CST3', 'BCL6', 'PAX5', 'CXCR5', 'TCL1A', 
                         'CD27', 'CD38', 'SDC1', 'PRDM1',
                         'IRF4', 'CXCR4', 'CD24', 'CR2', 
                         'IGHD', 'IGHM', 'IGHA1', 'IGHA2', 'IGHG1',
                         'IGHG2', 'IGHG3', 'IGHG4', 'IGHE', 'sample'],
          save='panel_bcell.pdf', ncols=4)

<Figure size 1545.6x2240 with 51 Axes>

In [22]:
sc.pl.dotplot(adata[adata.obs['site']=='Thymus'], var_names=['CD19', 'MS4A1', 'CD3E', 'CST3', 'BCL6', 'PAX5', 'CXCR5', 'TCL1A', 
                         'CD27', 'CD38', 'SDC1', 'PRDM1',
                         'IRF4', 'CXCR4', 'CD24', 'CR2', 
                         'IGHD', 'IGHM', 'IGHA1', 'IGHA2', 'IGHG1',
                         'IGHG2', 'IGHG3', 'IGHG4', 'IGHE'], groupby='leiden', swap_axes=True,
             save='thymus')

<Figure size 1896x780 with 4 Axes>

In [23]:
sc.pl.umap(adata[adata.obs['site']=='Thymus'], color=['CD19', 'MS4A1', 'CD3E', 'CST3', 'BCL6', 'PAX5', 'CXCR5', 'TCL1A', 
                         'CD27', 'CD38', 'SDC1', 'PRDM1',
                         'IRF4', 'CXCR4', 'CD24', 'CR2', 
                         'IGHD', 'IGHM', 'IGHA1', 'IGHA2', 'IGHG1',
                         'IGHG2', 'IGHG3', 'IGHG4', 'IGHE', 'sample'],
          save='panel_bcell.pdf', ncols=4)

<Figure size 1545.6x2240 with 51 Axes>

In [24]:
sc.pl.umap(adata[adata.obs['site']=='Thymus'], color=['leiden_R'],
          save='thymus_leiden.pdf', ncols=4)

<Figure size 320x320 with 1 Axes>

In [25]:
sc.pl.umap(adata[adata.obs['site']=='Periphery'], color=['leiden_R'],
          save='thymus_leiden.pdf', ncols=4)

<Figure size 320x320 with 1 Axes>

In [26]:
sc.tl.rank_genes_groups(adata, 'leiden_R', method='t-test_overestim_var')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:53)


<Figure size 1280x8320 with 102 Axes>

In [27]:
print('\n'.join(list(adata.uns['rank_genes_groups']['names']['7'])[:10]))

TCL1A
FCER2
ACTB
HLA-DRB5
VPREB3
CHI3L2
CD72
NCF1
RPL18A
MARCKSL1


In [28]:
plt.figure(figsize=(10,3))
sns.heatmap(pd.crosstab(adata.obs['site'], adata.obs['leiden']), cmap='viridis')

<AxesSubplot:xlabel='leiden', ylabel='site'>

<Figure size 800x240 with 2 Axes>

In [29]:
df_cross = pd.crosstab(adata.obs['site'], adata.obs['leiden'])
df_cross = (df_cross.T / df_cross.sum(axis=1)).T

plt.figure(figsize=(10,3))
sns.heatmap(df_cross, cmap='viridis')

<AxesSubplot:xlabel='leiden', ylabel='site'>

<Figure size 800x240 with 2 Axes>

In [30]:
c = list(adata.obs['leiden_R'].cat.categories)
df_clusters = pd.DataFrame(c, columns = ['cluster'])
df_clusters.index = df_clusters['cluster'].copy()
df_clusters.index.name = 'leiden_R'
df_clusters['cluster'] = 'B cell'
df_clusters.index = df_clusters.index.astype(str)
df_clusters.loc[['13', '28,3'], 'cluster'] = 'Doublet'
df_clusters

,cluster
leiden_R,
0,B cell
1,B cell
2,B cell
3,B cell
4,B cell
...,...
91,B cell
92,B cell
93,B cell


In [31]:
adata.obs['Bcell_all_cluster'] = [df_clusters.loc[x, 'cluster'] for x in adata.obs['leiden_R']]

In [32]:
with plt.rc_context({"figure.figsize": (5,5)}):
    sc.pl.umap(adata, color='Bcell_all_cluster', add_outline=True,
           title='', frameon=False, save='Bcell_all_cluster.pdf', s=30)

... storing 'Bcell_all_cluster' as categorical


<Figure size 400x400 with 1 Axes>

In [33]:
sc.pl.umap(adata, color='Bcell_all_cluster', title='', add_outline=True, 
           frameon=False, save='Bcell_all_cluster_out.pdf')

<Figure size 320x320 with 1 Axes>

In [35]:
adata.write(results_file_Bcell_all)

## select singlet B cells

In [36]:
adata = sc.read(results_file_Bcell_all)

In [37]:
adata_raw = sc.read(results_raw_file_master)

adata_cp = adata_raw[adata[adata.obs['Bcell_all_cluster']=='B cell'].obs.index].copy()
    
adata = adata_cp.copy()
adata_cp = None

adata = adata[adata.obs['sample'] != "MG03_PT"]

sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
adata.raw = sc.pp.log1p(adata, copy=True)

normalizing by total count per cell
Trying to set attribute `.obs` of view, copying.
    finished (0:00:03): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


In [38]:
adata.obs['sample']

AAAGGTATCCTGCTAC    MG21_TE
AAATGGAAGCGACAGT    MG21_TE
AACAAGACAACTGCTA    MG21_TE
AACCTGACAGGACTTT    MG21_TE
AACCTGAGTGAGACCA    MG21_TE
                     ...   
TTAGGGTTCGAAGAAT    MG23_TE
TTGCCTGAGGAAGTCC    MG23_TE
TTGTGTTTCGGAATTC    MG23_TE
TTGTTCACAGCTTTGA    MG23_TE
TTGTTGTCAAAGCTCT    MG23_TE
Name: sample, Length: 9770, dtype: category
Categories (8, object): ['MG03_PB', 'MG03_TE', 'MG03_TL', 'MG21_TE', 'MG21_TL', 'MG22_PL', 'MG22_TE', 'MG23_TE']

In [39]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [40]:
adata = adata[:,(~ adata.var.index.str.startswith('IGKV')) & 
              (~ adata.var.index.str.startswith('IGLV')) &
             (~ adata.var.index.str.startswith('IGHV')) &
             (~ adata.var.index.str.startswith('IGLC'))]

filter_result = sc.pp.filter_genes_dispersion(
    adata.X, min_mean=0.0125, max_mean=2.5, min_disp=0.7)
sc.pl.filter_genes_dispersion(filter_result)
print([sum([i[0] for i in filter_result]),len(filter_result)])

extracting highly variable genes
    finished (0:00:00)


<Figure size 640x320 with 2 Axes>

[823, 36265]


In [41]:
adata = adata[:, filter_result.gene_subset]

sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata)
adata.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
sc.pl.pca_variance_ratio(adata, log=True)

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:375: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
computing PCA
    with n_comps=50
    finished (0:00:00)


<Figure size 320x320 with 1 Axes>

In [42]:
sc.external.pp.bbknn(adata, batch_key='sample', n_pcs=50)
sc.tl.umap(adata, spread=1)
# sc.tl.leiden(adata, resolution=1)

computing batch balanced neighbors
	finished: added to `.uns['neighbors']`
	`.obsp['distances']`, distances for each pair of neighbors
	`.obsp['connectivities']`, weighted adjacency matrix (0:00:01)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:13)


In [43]:
sc.pl.umap(adata, color=['n_counts', 'n_genes', 'pct_counts_mt'], save='qc')

<Figure size 1159.2x320 with 6 Axes>

In [44]:
sc.tl.leiden(adata, resolution=1.4)

running Leiden clustering
    finished: found 17 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:02)


In [45]:
sc.pl.umap(adata, color='leiden', legend_loc='on data', title='', 
           frameon=False, save='Bcell_leiden.pdf')

<Figure size 320x320 with 1 Axes>

In [46]:
sc.pl.umap(adata[adata.obs['site']=='Thymus'], color=['leiden'],
          save='thymus_leiden.pdf', ncols=4)

<Figure size 320x320 with 1 Axes>

In [47]:
sc.pl.umap(adata[adata.obs['site']=='Periphery'], color=['leiden'],
          save='thymus_leiden.pdf', ncols=4)

<Figure size 320x320 with 1 Axes>

In [48]:
sc.tl.leiden(adata, resolution=0.6, restrict_to=('leiden', ['0']))

running Leiden clustering
    finished: found 21 clusters and added
    'leiden_R', the cluster labels (adata.obs, categorical) (0:00:00)


In [49]:
sc.pl.umap(adata[adata.obs['leiden']=='0'], color=['site', 'leiden_R'],
          save='thymus_leiden.pdf', ncols=4, size=20)

Trying to set attribute `.uns` of view, copying.


<Figure size 772.8x320 with 2 Axes>

In [50]:
df_cross = pd.crosstab(adata.obs['site'], adata.obs['leiden_R'])
df_cross = (df_cross.T / df_cross.sum(axis=1)).T

plt.figure(figsize=(10,3))
sns.heatmap(df_cross, cmap='viridis')

<AxesSubplot:xlabel='leiden_R', ylabel='site'>

<Figure size 800x240 with 2 Axes>

In [51]:
pd.crosstab(adata.obs['site'], adata.obs['leiden'])

leiden,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
site,,,,,,,,,,,,,,,,,
Periphery,1194,1080,1041,922,928,672,511,433,358,317,263,297,267,178,165,101,118
Thymus,199,137,65,36,19,8,33,21,66,70,77,3,25,61,61,35,9


In [52]:
sc.pl.umap(adata, color=['CCR7', 'JUN', 'GPR183', 'FCRL2', 'TNFRSF13B', 'IGHM', 'SSPN', 'TXNDC5', 'sample', 'site'],
          save='panel_bcell.pdf', ncols=4)

<Figure size 1545.6x960 with 18 Axes>

Antigen- activated naive B cells increase their metabolic activity17 and express
chemoattractant receptors (CC- chemokine receptor 7 (CCR7) and EBI2(GPR183)) that direct them to the border of the T cell zone

In [53]:
# TXNDC5 plasmablast, PC

sc.pl.umap(adata[adata.obs['site']=='Thymus'], 
           color=['CCR7', 'CD38', 'JUN', 'GPR183', 'FCRL2', 'PAX5', 'BACH2', 'TNFRSF13B', 'SSPN', 'TXNDC5', 'sample', 'site'],
          save='panel_bcell.pdf', ncols=4)

<Figure size 1545.6x960 with 22 Axes>

In [54]:
sc.pl.umap(adata, color=['IGHD', 'CD27', 'sample', 'site'],
          save='panel_bcell.pdf', ncols=4)

<Figure size 1545.6x320 with 6 Axes>

In [55]:
# GC related genes
# https://immunology.sciencemag.org/content/6/56/eabe6291

sc.pl.umap(adata[adata.obs['site']=='Thymus'], 
           color=['SPN', 'CD5', 'CD27', 'APEX1', 'XRCC5','BATF', 'IRF4', 'POLE3', 'sample', 'site'],
          save='panel_bcell.pdf', ncols=4)

<Figure size 1545.6x960 with 18 Axes>

In [56]:
# GC related genes
# https://immunology.sciencemag.org/content/6/56/eabe6291

sc.pl.umap(adata[adata.obs['site']=='Periphery'], 
           color=['IGHD', 'CD27', 'APEX1', 'XRCC5','BATF', 'IRF4', 'POLE3', 'sample', 'site'],
          save='panel_bcell.pdf', ncols=4)

<Figure size 1545.6x960 with 16 Axes>

In [57]:
sc.pl.umap(adata, color=['CD19', 'MS4A1', 'CD3E', 'MKI67', 'CST3', 'BCL6', 'PAX5', 'CXCR5', 'TCL1A', 
                         'CD27', 'CD38', 'SDC1', 'PRDM1', 'TXNDC5',
                         'IRF4', 'CXCR4', 'CD24', 'CR2', 'CD69',  
                         'IGHD', 'IGHM', 'IGHA1', 'IGHA2', 'IGHG1',
                         'IGHG2', 'IGHG3', 'IGHG4', 'IGHE', 'sample', 'site'],
          save='panel_bcell.pdf', ncols=4)

<Figure size 1545.6x2560 with 58 Axes>

In [58]:
sc.pl.dotplot(adata, var_names=['CD19', 'MS4A1', 'CD3E', 'CST3', 'BCL6', 'PAX5', 'CXCR5', 'TCL1A', 
                         'CD27', 'CD38', 'SDC1', 'PRDM1', 'XBP1',
                         'FKBP11', 
                         'IRF4', 'CXCR4', 'CD24', 'CR2', 
                         'IGHD', 'IGHM', 'IGHA1', 'IGHA2', 'IGHG1',
                         'IGHG2', 'IGHG3', 'IGHG4', 'IGHE'], 
              groupby='leiden_R', swap_axes=True, dendrogram=True)

    using 'X_pca' with n_pcs = 50
Storing dendrogram info using `.uns['dendrogram_leiden_R']`


<Figure size 741.6x900 with 5 Axes>

In [59]:
sc.tl.rank_genes_groups(adata, 'leiden_R', method='t-test_overestim_var')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:05)


<Figure size 1280x1920 with 21 Axes>

In [60]:
print('\n'.join(list(adata.uns['rank_genes_groups']['names']['15'])[:100]))

SOX4
CD3D
NUCB2
MZB1
CD3G
TRBC1
IGLC2
FKBP11
BCL11B
XBP1
IL7R
MAL
FYB1
LEF1
FOS
CD3E
HSPA1B
CD7
CHPF
H1FX
CD2
PRDX4
SSR4
AIF1
PRDM1
YBX3
ITM2C
MANF
IGLC3
SDC1
PTCRA
JCHAIN
SH2D1A
CD38
AQP3
GIHCG
FXYD2
STMN1
CDK6
SELENOS
SPAG4
SRGN
TXNDC5
TFDP2
SLAMF7
TCF7
HSPA1A
ERN1
CD8B
TRAT1
DERL3
JSRP1
TPST2
IGHG4
TRAC
SSR3
CD1E
NT5DC2
IGHM
LDLRAD4
IL32
TNFRSF17
FNDC3B
MAP1A
ITM2A
ERLEC1
SMIM3
TCL1A
SEC11C
HNRNPH1
DSP
DBNDD1
SSR1
HERPUD1
ID2
SEL1L
TMEM263
ITGA6
SCNN1B
PCDH9
TMEM258
TRIB1
DUSP1
CD1B
UGP2
ANXA1
SOCS2
IGHG3
NANS
TRDC
TMED4
GZMM
LARP1B
FOSB
INSR
ABHD17B
GADD45A
CD9
IL6ST
SEC61G


In [61]:
# 'PRDM1', 'XBP1', 'MZB1' : plasmablast
# 'BCL2A1' : LZ GC
# 'EZR' : DZ GC
list_gc_markers = ['FCER2', 'SELL', 'BANK1', 'CD69', 'JUN', 'BHLHE40', 'BCL2A1' ,
                   'EZR', 'PRDM1', 'XBP1', 'MZB1',
                  'CCR1', 'ITGAX', 'MKI67']
sc.pl.umap(adata[adata.obs['site']=='Thymus'], color=list_gc_markers,
          save='thymus_panel_gcbcell.pdf', ncols=4)

<Figure size 1545.6x1280 with 28 Axes>

In [62]:
sc.pl.umap(adata[adata.obs['site']=='Periphery'], color=list_gc_markers,
          save='peri_panel_gcbcell.pdf', ncols=4)

<Figure size 1545.6x1280 with 28 Axes>

In [63]:
sc.pl.umap(adata[adata.obs['site']=='Thymus'], color=['leiden'],
          save='thymus_leiden.pdf', ncols=4)

<Figure size 320x320 with 1 Axes>

In [64]:
sc.pl.umap(adata[adata.obs['site']=='Periphery'], color=['leiden'],
          save='thymus_leiden.pdf', ncols=4)

<Figure size 320x320 with 1 Axes>

In [65]:
sc.pl.dotplot(adata, var_names=list_gc_markers, 
              groupby='leiden', swap_axes=True)

<Figure size 623.2x472 with 4 Axes>

In [66]:
sc.pl.dotplot(adata[adata.obs['site']=='Thymus'], var_names=list_gc_markers, 
              groupby='leiden', swap_axes=True)

<Figure size 623.2x472 with 4 Axes>

In [67]:
sc.pl.umap(adata[adata.obs['site']=='Thymus'], color=['CD19', 'MS4A1', 'CD3E', 'CST3', 'BCL6', 'PAX5', 'CXCR5', 'TCL1A', 
                         'CD27', 'CD38', 'SDC1', 'PRDM1',
                         'IRF4', 'CXCR4', 'CD24', 'CR2', 
                         'IGHD', 'IGHM', 'IGHA1', 'IGHA2', 'IGHG1',
                         'IGHG2', 'IGHG3', 'IGHG4', 'IGHE', 'sample'],
          save='thymus_panel_bcell.pdf', ncols=4)

<Figure size 1545.6x2240 with 51 Axes>

In [68]:
sc.pl.umap(adata[adata.obs['site']=='Periphery'], color=['leiden_R'], legend_loc='on data',
          save='Periphery_leiden.pdf', ncols=4)

Trying to set attribute `.uns` of view, copying.


<Figure size 320x320 with 1 Axes>

In [69]:
sc.pl.umap(adata[adata.obs['leiden_R'].isin(['8', '10', '15'])], color=['CD3E', 'IGHD', 'CD27','CD24', 'CD38', 'MZB1', 'leiden_R', 'site'], legend_loc='on data',
          save=False, ncols=4)

Trying to set attribute `.uns` of view, copying.


<Figure size 1545.6x640 with 14 Axes>

In [70]:
sc.pl.dotplot(adata[adata.obs['leiden_R'].isin(['8', '10', '15'])], 
              ['CD3E','MS4A1','IGHD', 'CD27','CD24', 'CD38', 'MZB1', "IGHG1", "IGHG2", 'IGHM', 'IGHD'], groupby="leiden_R")

<Figure size 445.6x200 with 4 Axes>

In [71]:
sc.pl.umap(adata[adata.obs['site']=='Periphery'], color=list_gc_markers,
          save='peri_panel_gcbcell.pdf', ncols=4)

<Figure size 1545.6x1280 with 28 Axes>

In [72]:
sc.pl.umap(adata[adata.obs['site']=='Periphery'], color=['CD19', 'MS4A1', 'CD3E', 'CST3', 'BCL6', 'PAX5', 'CXCR5', 'TCL1A', 
                         'CD27', 'CD38', 'SDC1', 'PRDM1',
                         'IRF4', 'CXCR4', 'CD24', 'CR2', 
                         'IGHD', 'IGHM', 'IGHA1', 'IGHA2', 'IGHG1',
                         'IGHG2', 'IGHG3', 'IGHG4', 'IGHE', 'sample'],
          save='peri_panel_bcell.pdf', ncols=4)

<Figure size 1545.6x2240 with 51 Axes>

In [73]:
for c in adata.obs['leiden_R'].cat.categories:
    sc.pl.umap(adata, color=list(adata.uns['rank_genes_groups']['names'][c])[:20],
          save='B_leiden{}'.format(c), ncols=5)

<Figure size 1932x1280 with 40 Axes>

<Figure size 1932x1280 with 40 Axes>

<Figure size 1932x1280 with 40 Axes>

<Figure size 1932x1280 with 40 Axes>

<Figure size 1932x1280 with 40 Axes>

...

<Figure size 1932x1280 with 40 Axes>

<Figure size 1932x1280 with 40 Axes>

<Figure size 1932x1280 with 40 Axes>

<Figure size 1932x1280 with 40 Axes>

<Figure size 1932x1280 with 40 Axes>

In [74]:
ax = sc.pl.correlation_matrix(adata, 'leiden_R', figsize=(7,7))

<Figure size 560x560 with 3 Axes>

In [75]:
sc.tl.paga(adata, groups='leiden')
sc.pl.paga_compare(adata, threshold=0.7, basis='umap')

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)
--> added 'pos', the PAGA positions (adata.uns['paga'])


<Figure size 595.36x320 with 2 Axes>

In [76]:
a_t = adata[adata.obs['site']=='Thymus'].copy()
sc.tl.paga(a_t, groups='leiden')
sc.pl.paga_compare(a_t, threshold=0.3, basis='umap')

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)
--> added 'pos', the PAGA positions (adata.uns['paga'])


<Figure size 595.36x320 with 2 Axes>

In [77]:
df_clusters =  pd.read_csv('210430_clusters/Bcell_clusters.csv', index_col=0)
df_clusters.index = df_clusters.index.astype(str)
adata.obs['Bcell_cluster'] = [df_clusters.loc[x, 'cluster'] for x in adata.obs['leiden_R']]
df_clusters

,cluster
leiden,
"0,0",Memory B cell (I)
"0,1",Memory B cell (I)
"0,2",Thymic memory B cell
"0,3",Memory B cell (I)
"0,4",Memory B cell (I)
1,Naive B cell
2,Unswitched memory B cell
3,Naive B cell
4,Naive B cell


In [78]:
with plt.rc_context({"figure.figsize": (5,5)}):
    sc.pl.umap(adata, color='Bcell_cluster', add_outline=True,
           title='', frameon=False, save='Bcell_cluster.pdf') 

... storing 'Bcell_cluster' as categorical


<Figure size 400x400 with 1 Axes>

In [79]:
sc.pl.umap(adata, color='Bcell_cluster', title='', add_outline=True, 
           frameon=False, save='Bcell_cluster_out.pdf')

<Figure size 320x320 with 1 Axes>

In [80]:
sc.tl.rank_genes_groups(adata, 'Bcell_cluster', method='t-test_overestim_var')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)


<Figure size 1280x640 with 8 Axes>

In [81]:
sc.pl.umap(adata, color=list(adata.uns['rank_genes_groups']['names']['GC B cell'])[:20])

<Figure size 1545.6x1600 with 40 Axes>

In [82]:
print('\n'.join(list(adata.uns['rank_genes_groups']['names']['GC B cell'])[:100]))

LMO2
MARCKSL1
MEF2B
RGS13
BASP1
KLHL6
SYNE2
LRMP
CCDC144A
ACTB
TCL1A
NEIL1
LPP
RFTN1
SUGCT
ACTG1
DBI
DAAM1
HMCES
SERF2
MYBL1
SERPINA9
PAG1
TMEM123
GCHFR
AC023590.1
STAG3
ACY3
GCSAM
TMSB4X
GMDS
CD38
MME
TOX
NANS
BCL6
LINC00877
FAM126A
HOPX
SORL1
METAP2
RASL11A
NUGGC
HLA-DMB
ARPC2
TNFRSF17
SMIM14
CD22
GSTP1
LCK
CORO1A
DHRS9
ARPC5L
RGS10
A4GALT
DTX1
SPRED2
PRDX6
ATP5MG
MIR3681HG
ELL3
CLTC
HMGN1
SOCS1
RAPGEF5
BIK
IRF8
CDCA7
PARP1
TKT
ACTR3
CD40
CASP3
LBR
CCDC88A
OGG1
DEF8
CAMK1
SPATS2
PTTG1
ROMO1
ARPC5
FAM3C
TBC1D1
RGS16
KTN1
MED12L
MEF2C
PLEKHA2
ATP5MC3
PDIA6
LIMS1
IL4R
POMP
S1PR2
BLOC1S6
BRK1
ADA
MYO1E
ANP32B


In [83]:
sc.tl.rank_genes_groups(adata, 'Bcell_cluster', method='t-test_overestim_var', reference='Memory B cell (I)', 
                        groups=['Memory B cell (II)'])
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


<Figure size 320x320 with 1 Axes>

In [84]:
sc.pl.dotplot(adata, list(adata.uns['rank_genes_groups']['names']['Memory B cell (II)'])[:40], groupby='Bcell_cluster')

<Figure size 1304x304 with 4 Axes>

In [85]:
sc.tl.rank_genes_groups(adata, 'Bcell_cluster', method='t-test_overestim_var')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)


<Figure size 1280x640 with 8 Axes>

In [86]:
sc.tl.dendrogram(adata, groupby='Bcell_cluster')
sc.pl.rank_genes_groups_dotplot(adata, groupby='Bcell_cluster', n_genes=15, save='Bcell_cluster.pdf')

    using 'X_pca' with n_pcs = 50
Storing dendrogram info using `.uns['dendrogram_Bcell_cluster']`


<Figure size 3736x304 with 6 Axes>

In [87]:
sc.pl.dotplot(adata, ['PSME2', 'BHLHE40', 'PARVB'], groupby='Bcell_cluster')

<Figure size 208.8x304 with 4 Axes>

In [88]:
ax = sc.pl.correlation_matrix(adata, 'Bcell_cluster', figsize=(5,3.5))

<Figure size 400x280 with 3 Axes>

In [89]:
sc.tl.paga(adata, groups='Bcell_cluster')
sc.pl.paga_compare(adata, threshold=0.7, basis='X_umap')

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)
--> added 'pos', the PAGA positions (adata.uns['paga'])


<Figure size 595.36x320 with 2 Axes>

In [90]:
sc.pl.paga(adata, threshold=0.5)

--> added 'pos', the PAGA positions (adata.uns['paga'])


<Figure size 297.68x320 with 1 Axes>

In [92]:
sc.pl.umap(adata, 
           color=['sample', 'leiden'], 
           save=False)

<Figure size 772.8x320 with 2 Axes>

In [93]:
plt.figure(figsize=(10,3))
sns.heatmap(pd.crosstab(adata.obs['site'], adata.obs['Bcell_cluster']), cmap='viridis')

<AxesSubplot:xlabel='Bcell_cluster', ylabel='site'>

<Figure size 800x240 with 2 Axes>

In [94]:
df_cross = pd.crosstab(adata.obs['site'], adata.obs['Bcell_cluster'])
df_cross = (df_cross.T / df_cross.sum(axis=1)).T

plt.figure(figsize=(10,3))
sns.heatmap(df_cross, cmap='viridis')

<AxesSubplot:xlabel='Bcell_cluster', ylabel='site'>

<Figure size 800x240 with 2 Axes>

In [95]:
df_cross = pd.crosstab(adata.obs['site'], adata.obs['Bcell_cluster'])
df_cross = (df_cross.T / df_cross.sum(axis=1)).T

plt.figure(figsize=(10,3))
sns.heatmap(df_cross, cmap='viridis', annot=True)

<AxesSubplot:xlabel='Bcell_cluster', ylabel='site'>

<Figure size 800x240 with 2 Axes>

In [96]:
plt.figure(figsize=(4,3))
df_cross.loc['Thymus'].plot.bar()
plt.grid(False)

<Figure size 320x240 with 1 Axes>

In [99]:
d = adata.obs[adata.obs['site'] == 'Thymus']
d['individual'] = d['individual'].str.split('_').str.get(0)
d = pd.crosstab(d['individual'], d['Bcell_cluster'])
d = d.T / d.sum(axis=1)

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [100]:
sns.heatmap(d, cmap='viridis', annot=True)

<AxesSubplot:xlabel='individual', ylabel='Bcell_cluster'>

<Figure size 320x320 with 2 Axes>

In [101]:
d = adata.obs[adata.obs['site'] == 'Periphery']
d['individual'] = d['individual'].str.split('_').str.get(0)
d = pd.crosstab(d['individual'], d['Bcell_cluster'])
d = d.T / d.sum(axis=1)
sns.heatmap(d, cmap='viridis', annot=True)

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<AxesSubplot:xlabel='individual', ylabel='Bcell_cluster'>

<Figure size 320x320 with 2 Axes>

In [99]:
sc.pl.stacked_violin(adata,['IGHM', 'IGHD', 'IGHA1', 'IGHA2', 'IGHG1', 'IGHG2', 'IGHG3', 'IGHG4', 'IGHE'],
                    groupby='Bcell_cluster', save='ig')

<Figure size 386.4x304 with 11 Axes>

In [103]:
adata

AnnData object with n_obs × n_vars = 9770 × 823
    obs: 'sample', 'sample_type', 'site', 'fraction', 'n_genes', 'individual', 'score_DEGs', 'score_yellow', 'score_GWAS', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'leiden_R', 'Bcell_cluster'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: 'log1p', 'pca', 'neighbors', 'umap', 'leiden', 'leiden_colors', 'sample_colors', 'site_colors', 'dendrogram_leiden_R', 'rank_genes_groups', 'paga', 'leiden_sizes', 'Bcell_cluster_colors', 'dendrogram_Bcell_cluster', 'Bcell_cluster_sizes'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [100]:
adata.write(results_file_Bcell)

## summarize clusters

In [101]:
adata_b_all = sc.read(results_file_Bcell_all)
adata_b = sc.read(results_file_Bcell)

In [102]:
df_cluster = adata_b_all.obs[['Bcell_all_cluster']]
df_cluster.columns = ['minor_cluster']
df_cluster['minor_cluster'] = df_cluster['minor_cluster'].astype(str)
df_cluster.head()

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,minor_cluster
AAAGGTATCCTGCTAC,B cell
AAATGGAAGCGACAGT,B cell
AACAAGACAACTGCTA,B cell
AACCTGACAGGACTTT,B cell
AACCTGAGTGAGACCA,B cell


In [103]:
df_cluster_bcell = adata_b.obs[['Bcell_cluster']]
df_cluster_bcell.columns = ['minor_cluster']
df_cluster_bcell['minor_cluster'] = df_cluster_bcell['minor_cluster'].astype(str)
df_cluster_bcell.head()

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,minor_cluster
AAAGGTATCCTGCTAC,Unswitched memory B cell
AAATGGAAGCGACAGT,Naive B cell
AACAAGACAACTGCTA,Naive B cell
AACCTGACAGGACTTT,Unswitched memory B cell
AACCTGAGTGAGACCA,Thymic memory B cell


In [104]:
df_cluster.loc[df_cluster_bcell.index, 'minor_cluster'] = list(df_cluster_bcell['minor_cluster'])
adata_b_all.obs['minor_cluster'] = list(df_cluster['minor_cluster'])

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [105]:
adata_b_all = adata_b_all[adata_b_all.obs['minor_cluster'] !='B cell']

In [106]:
sc.pl.umap(adata_b_all, 
           color=['minor_cluster'], 
           save='minor_cluster')

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/anndata/_core/anndata.py:1229: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning
Trying to set attribute `.obs` of view, copying.
... storing 'minor_cluster' as categorical


<Figure size 320x320 with 1 Axes>

In [107]:
sc.pl.umap(adata_b_all, color='minor_cluster', title='', add_outline=True, 
           frameon=False, save='minor_cluster_out.pdf')

<Figure size 320x320 with 1 Axes>

In [108]:
adata_b_all.write(results_file_Bcell_minor_cluster)

In [109]:
adata = sc.read(results_file_Bcell_minor_cluster)
adata_raw = sc.read(results_raw_file_master)

adata_cg = adata_raw[adata.obs.index].copy()
adata_cg.obs = adata.obs
adata_cg.obsm = adata.obsm
adata_cg.uns = adata.uns

sc.pp.normalize_per_cell(adata_cg, counts_per_cell_after=1e4)
sc.pp.log1p(adata_cg)

adata_cg.write(results_file_cg_Bcell_minor_cluster)

normalizing by total count per cell
    finished (0:00:01): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


## visualize

In [110]:
adata = sc.read(results_file_Bcell)
adata.obs['individual'] = adata.obs['individual'].str.split('_').str.get(0)

In [111]:
with plt.rc_context({"figure.figsize": (5,5)}):
    sc.pl.umap(adata, color='Bcell_cluster', add_outline=True,
           title='Thymus + Periphery', frameon=False, save='Bcell_cluster.pdf') 

... storing 'individual' as categorical


<Figure size 400x400 with 1 Axes>

In [112]:
with plt.rc_context({"figure.figsize": (5,5)}):
    sc.pl.umap(adata[adata.obs['site'] == 'Thymus'], color='Bcell_cluster', add_outline=True,
           title='Thymus', frameon=False, save='Bcell_cluster_thymus.pdf') 

<Figure size 400x400 with 1 Axes>

In [113]:
with plt.rc_context({"figure.figsize": (5,5)}):
    sc.pl.umap(adata[adata.obs['site'] == 'Periphery'], color='Bcell_cluster', add_outline=True,
           title='Periphery', frameon=False, save='Bcell_cluster_peri.pdf') 

<Figure size 400x400 with 1 Axes>

In [114]:
sc.tl.embedding_density(adata, groupby='site')

computing density on 'umap'
--> added
    'umap_density_site', densities (adata.obs)
    'umap_density_site_params', parameter (adata.uns)


In [115]:
sc.pl.embedding_density(adata, groupby='site', bg_dotsize=40, fg_dotsize=50, save='site')

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/scanpy/plotting/_tools/__init__.py:1186: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("YlOrRd").copy()
  color_map.set_over('black')
/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/scanpy/plotting/_tools/__init__.py:1187: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("YlOrRd").copy()
  color_map.set_under('lightgray')
/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/scanpy/plotting/_too

<Figure size 772.8x320 with 4 Axes>

In [116]:
for site in ['Periphery', 'Thymus']:
    sc.pl.umap(adata, color='site', groups=[site])

<Figure size 320x320 with 1 Axes>

<Figure size 320x320 with 1 Axes>

In [120]:
list_label = ['Naive B cell', 
              'Pre GC B cell', 'GC B cell', 'Thymic memory B cell', 'Unswitched memory B cell',
              'Memory B cell (I)', 'Memory B cell (II)', 'Plasmablast']

In [122]:
df_cross = pd.crosstab(adata.obs['site'], adata.obs['Bcell_cluster'])
df_cross = (df_cross.T / df_cross.sum(axis=1)).T

plt.figure(figsize=(10,3))
sns.heatmap(df_cross[list_label], cmap='viridis', annot=False)

<AxesSubplot:xlabel='Bcell_cluster', ylabel='site'>

<Figure size 800x240 with 2 Axes>

In [123]:
df_cross = pd.crosstab(adata.obs['site'], adata.obs['Bcell_cluster'])

In [124]:
import scipy.stats as stats
import statsmodels.stats.multitest as multi

list_pvalue = []
for label in list_label:
    a = df_cross.loc['Periphery', label]
    b = df_cross.loc['Thymus', label]
    c = df_cross.loc['Periphery', list(set(list_label) - set(label))].sum()
    d = df_cross.loc['Thymus', list(set(list_label) - set(label))].sum()
    
    oddsratio, pvalue = stats.fisher_exact([[a, b],[c, d]])
    list_pvalue.append(pvalue)
    
multi.fdrcorrection(list_pvalue)

(array([ True,  True,  True,  True, False,  True,  True,  True]),
 array([5.50789955e-11, 4.02771167e-07, 3.85358434e-13, 1.02410136e-61,
        7.84085083e-01, 3.02586592e-11, 7.10768488e-03, 4.82671815e-12]))

In [125]:
pd.crosstab(adata.obs['site'], adata.obs['Bcell_cluster'], normalize=0).T.loc[list_label].plot.bar(figsize=(4,3))
# plt.yscale('log')
plt.grid(None)

plt.savefig(str(sc.settings.figdir) + '/bar_site.pdf', bbox_inches='tight')

<Figure size 320x240 with 1 Axes>

In [126]:
adata.obs.value_counts('sample')

sample
MG03_PB    8683
MG21_TL     555
MG21_TE     227
MG22_PL     162
MG23_TE      74
MG03_TE      49
MG22_TE      12
MG03_TL       8
dtype: int64

In [127]:
import scipy.stats as stats
import statsmodels.stats.multitest as multi

list_prop = []

d = adata.obs[adata.obs['site'] == 'Thymus']
d['individual'] = d['individual'].str.split('_').str.get(0)
d = pd.crosstab(d['individual'], d['Bcell_cluster'])
d = d[list_label]
d = d.T / d.sum(axis=1)
d = d.reset_index().melt(id_vars='Bcell_cluster')
d['site'] = 'Thymus'

list_prop.append(d)

d = adata.obs[adata.obs['site'] == 'Periphery']
d['individual'] = d['individual'].str.split('_').str.get(0)
d = pd.crosstab(d['individual'], d['Bcell_cluster'])
d = d[list_label]
d = d.T / d.sum(axis=1)
d = d.reset_index().melt(id_vars='Bcell_cluster')
d['site'] = 'Periphery'
list_prop.append(d)

df_prop = pd.concat(list_prop)

/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/yyasumizu/anaconda3/envs/scanpy1.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [128]:
list_p = []
for lab in list_label:
    _, p = stats.ttest_ind(df_prop.loc[(df_prop['site'] == 'Thymus') & (df_prop['Bcell_cluster'] == lab), 'value'],
                      df_prop.loc[(df_prop['site'] == 'Periphery') & (df_prop['Bcell_cluster'] == lab), 'value'])
    list_p.append(p)
multi.fdrcorrection(list_p)

(array([False, False, False, False, False, False, False, False]),
 array([0.83704014, 0.83704014, 0.89952848, 0.89952848, 0.83704014,
        0.83704014, 0.83704014, 0.83704014]))

In [129]:
plt.figure(figsize=(6,3))
# sns.swarmplot(data=df_prop, x='CD4T_cluster', y='value', hue='site')
sns.stripplot(data=df_prop, x='Bcell_cluster', y='value', hue='site', order=list_label)
plt.xticks(rotation=90)
plt.savefig(str(sc.settings.figdir) + '/stripplot_site.pdf', bbox_inches='tight')

<Figure size 480x240 with 1 Axes>

In [130]:
df_prop[df_prop['Bcell_cluster']=='GC B cell']

,Bcell_cluster,individual,value,site
2,GC B cell,MG03,0.000000,Thymus
10,GC B cell,MG21,0.078005,Thymus
18,GC B cell,MG22,0.000000,Thymus
26,GC B cell,MG23,0.000000,Thymus
2,GC B cell,MG03,0.018772,Periphery
10,GC B cell,MG22,0.012346,Periphery


In [131]:
adata[adata.obs['Bcell_cluster'] == 'GC B cell'].obs['sample'].value_counts()

MG03_PB    163
MG21_TL     56
MG21_TE      5
MG22_PL      2
Name: sample, dtype: int64

In [132]:
adata.obs['sample'].value_counts()

MG03_PB    8683
MG21_TL     555
MG21_TE     227
MG22_PL     162
MG23_TE      74
MG03_TE      49
MG22_TE      12
MG03_TL       8
Name: sample, dtype: int64

In [188]:
import pymc3 as pm
import arviz as az

print(f"Running on PyMC3 v{pm.__version__}")

Running on PyMC3 v3.11.2


In [189]:
donors = adata.obs.individual.unique()

x_t = []
x_b = []
n_t = []
n_b = []
for lab in list_label:
    _x_t = []
    _x_b = []
    _n_t = []
    _n_b = []
    for d in donors:
        try:
            _x_t.append(adata[(adata.obs['Bcell_cluster'] == lab) & (adata.obs['site'] == 'Thymus')
                            ].obs['individual'].value_counts()[d])
        except:
            _x_t.append(0)
            
        try:
            _x_b.append(adata[(adata.obs['Bcell_cluster'] == lab) & (adata.obs['site'] == 'Periphery')
                            ].obs['individual'].value_counts()[d])
        except:
            _x_b.append(0)
            
        try:
            _n_t.append(adata[adata.obs['site'] == 'Thymus'].obs['individual'].value_counts()[d])
        except:
            _n_t.append(0)
            
        try:
            _n_b.append(adata[adata.obs['site'] == 'Periphery'].obs['individual'].value_counts()[d])
        except:
            _n_b.append(0)
    x_t.append(_x_t)
    x_b.append(_x_b)
    n_t.append(_n_t)
    n_b.append(_n_b)

x_t = np.array(x_t).T
x_b = np.array(x_b).T
n_t = np.array(n_t).T
n_b = np.array(n_b).T

In [190]:
with pm.Model() as model:
    p_thymus = pm.Uniform("p_thymus", lower=0.0, upper=1.0, shape=len(list_label))
    p_blood = pm.Uniform("p_blood", lower=0.0, upper=1.0, shape=len(list_label))
    x_thymus = pm.Binomial("x_thymus", p=p_thymus, 
                           observed=x_t, 
                           n=n_t, shape=len(list_label))
    x_blood= pm.Binomial("x_blood", p=p_blood, observed=x_b, 
                         n=n_b, shape=len(list_label))
    
    p = pm.Deterministic("p", p_thymus - p_blood)

In [191]:
model

In [232]:
with model:
    trace = pm.sample(25000, chains=4)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p_blood, p_thymus]


Sampling 4 chains for 1_000 tune and 25_000 draw iterations (4_000 + 100_000 draws total) took 25 seconds.


In [233]:
pm.traceplot(trace, var_names=["p"]);

<Figure size 960x160 with 2 Axes>

In [234]:
pm.plot_posterior(
    trace,
    var_names=["p"],
    ref_val = 0
);

<Figure size 1472x736 with 8 Axes>

In [235]:
pm.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
p_thymus[0],0.361,0.016,0.332,0.391,0.0,0.0,136130.0,77697.0,1.0
p_thymus[1],0.077,0.009,0.061,0.093,0.0,0.0,132623.0,77080.0,1.0
p_thymus[2],0.067,0.008,0.052,0.082,0.0,0.0,141591.0,75982.0,1.0
p_thymus[3],0.175,0.012,0.152,0.198,0.0,0.0,138983.0,75272.0,1.0
p_thymus[4],0.164,0.012,0.141,0.187,0.0,0.0,125838.0,73620.0,1.0
p_thymus[5],0.042,0.007,0.030,0.054,0.0,0.0,132841.0,74348.0,1.0
p_thymus[6],0.037,0.006,0.025,0.048,0.0,0.0,130789.0,75342.0,1.0
p_thymus[7],0.084,0.009,0.067,0.101,0.0,0.0,137429.0,74883.0,1.0
p_blood[0],0.555,0.005,0.545,0.565,0.0,0.0,141968.0,78130.0,1.0
p_blood[1],0.036,0.002,0.032,0.040,0.0,0.0,142185.0,74696.0,1.0


In [236]:
pm.forestplot(trace)

array([<AxesSubplot:title={'center':'94.0% HDI'}>], dtype=object)

<Figure size 480x1248 with 1 Axes>

In [237]:
ax = az.plot_forest(trace, var_names=["p"])
ax[0].set_yticklabels([list_label[-i] for i in range(len(list_label))]);

<Figure size 480x576 with 1 Axes>

In [238]:
p_thymus = []
p_blood = []

for i in range(len(list_label)):
    p_thymus.append((trace['p'][:,i] < 0).sum() / trace['p'].shape[0])
    p_blood.append((trace['p'][:,i] > 0).sum() / trace['p'].shape[0])

In [239]:
df_stats = pd.DataFrame([p_thymus, p_blood], index=['Thymus', 'Peripheral'], columns=list_label).T
df_stats

,Thymus,Peripheral
Naive B cell,1.00000,0.00000
Pre GC B cell,0.00000,1.00000
GC B cell,0.00000,1.00000
Thymic memory B cell,0.00000,1.00000
Unswitched memory B cell,0.63233,0.36767
Memory B cell (I),1.00000,0.00000
Memory B cell (II),0.99815,0.00185
Plasmablast,0.00000,1.00000


In [240]:
df_summary = pm.summary(trace, hdi_prob=0.98)
df_summary

,mean,sd,hdi_1%,hdi_99%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
p_thymus[0],0.361,0.016,0.324,0.397,0.0,0.0,136130.0,77697.0,1.0
p_thymus[1],0.077,0.009,0.057,0.097,0.0,0.0,132623.0,77080.0,1.0
p_thymus[2],0.067,0.008,0.049,0.087,0.0,0.0,141591.0,75982.0,1.0
p_thymus[3],0.175,0.012,0.146,0.204,0.0,0.0,138983.0,75272.0,1.0
p_thymus[4],0.164,0.012,0.136,0.192,0.0,0.0,125838.0,73620.0,1.0
p_thymus[5],0.042,0.007,0.028,0.058,0.0,0.0,132841.0,74348.0,1.0
p_thymus[6],0.037,0.006,0.023,0.051,0.0,0.0,130789.0,75342.0,1.0
p_thymus[7],0.084,0.009,0.064,0.106,0.0,0.0,137429.0,74883.0,1.0
p_blood[0],0.555,0.005,0.543,0.567,0.0,0.0,141968.0,78130.0,1.0
p_blood[1],0.036,0.002,0.031,0.040,0.0,0.0,142185.0,74696.0,1.0


In [241]:
df_summary_thymus = df_summary.loc[[x for x in df_summary.index if 'p_thymus' in x]]
df_summary_thymus.index = list_label

df_summary_blood = df_summary.loc[[x for x in df_summary.index if 'p_blood' in x]]
df_summary_blood.index = list_label

In [242]:
df_summary_thymus

,mean,sd,hdi_1%,hdi_99%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Naive B cell,0.361,0.016,0.324,0.397,0.0,0.0,136130.0,77697.0,1.0
Pre GC B cell,0.077,0.009,0.057,0.097,0.0,0.0,132623.0,77080.0,1.0
GC B cell,0.067,0.008,0.049,0.087,0.0,0.0,141591.0,75982.0,1.0
Thymic memory B cell,0.175,0.012,0.146,0.204,0.0,0.0,138983.0,75272.0,1.0
Unswitched memory B cell,0.164,0.012,0.136,0.192,0.0,0.0,125838.0,73620.0,1.0
Memory B cell (I),0.042,0.007,0.028,0.058,0.0,0.0,132841.0,74348.0,1.0
Memory B cell (II),0.037,0.006,0.023,0.051,0.0,0.0,130789.0,75342.0,1.0
Plasmablast,0.084,0.009,0.064,0.106,0.0,0.0,137429.0,74883.0,1.0


In [243]:
df_summary_blood

,mean,sd,hdi_1%,hdi_99%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Naive B cell,0.555,0.005,0.543,0.567,0.0,0.0,141968.0,78130.0,1.0
Pre GC B cell,0.036,0.002,0.031,0.040,0.0,0.0,142185.0,74696.0,1.0
GC B cell,0.019,0.001,0.016,0.022,0.0,0.0,144062.0,75693.0,1.0
Thymic memory B cell,0.023,0.002,0.020,0.027,0.0,0.0,138853.0,75524.0,1.0
Unswitched memory B cell,0.168,0.004,0.159,0.177,0.0,0.0,142264.0,77256.0,1.0
Memory B cell (I),0.112,0.003,0.104,0.120,0.0,0.0,142804.0,75944.0,1.0
Memory B cell (II),0.058,0.002,0.052,0.064,0.0,0.0,132214.0,76107.0,1.0
Plasmablast,0.030,0.002,0.026,0.034,0.0,0.0,136444.0,76769.0,1.0


In [245]:
plt.figure(figsize=(4,2.5))
plt.scatter(np.array(range(len(list_label)))-0.1, df_summary_thymus['mean'], c='tab:blue', label='Thymus')
plt.scatter(np.array(range(len(list_label)))+0.1, df_summary_blood['mean'], c='tab:orange', label='Periphery')

for i,lab in enumerate(list_label):
    plt.plot([i-0.1, i-0.1],[df_summary_thymus.loc[lab,'hdi_1%'],df_summary_thymus.loc[lab,'hdi_99%']], c='tab:blue')
    plt.plot([i+0.1, i+0.1],[df_summary_blood.loc[lab,'hdi_1%'],df_summary_blood.loc[lab,'hdi_99%']], c='tab:orange')
    
plt.xticks(ticks=range(len(list_label)), labels=list_label, rotation=90)
plt.ylabel('Cell propotion')
plt.legend()
plt.savefig(str(sc.settings.figdir) + '/forest_hdi1%99%.pdf', bbox_inches='tight')

<Figure size 320x200 with 1 Axes>

In [147]:
sc.tl.paga(adata, groups='Bcell_cluster')

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)


In [148]:
sc.pl.paga(adata, threshold=0.5, save='bcell_cluster')

--> added 'pos', the PAGA positions (adata.uns['paga'])


<Figure size 297.68x320 with 1 Axes>

In [117]:
list_ig = ['IGHM', 'IGHD', 'IGHA1', 'IGHA2', 'IGHG1', 'IGHG2', 'IGHG3', 'IGHG4', 'IGHE']
sc.tl.dendrogram(adata, 'Bcell_cluster', var_names=list_ig)
sc.pl.matrixplot(adata, list_ig, 
              groupby='Bcell_cluster', dendrogram=True, save='Igclass')

Storing dendrogram info using `.uns['dendrogram_Bcell_cluster']`


<Figure size 450.4x304 with 4 Axes>

In [23]:
df_marker = pd.read_csv('./210430_clusters/markergenes_manual.csv', index_col=0)
df_marker = df_marker.loc[list_label]

dict_markers = {pos:[row[0], row[1], row[2], row[3], row[4]] for pos,row in df_marker.iterrows()}

In [24]:
sc.pl.dotplot(adata, dict_markers, 'Bcell_cluster', categories_order=list_label, dendrogram=False, save='minor_manual_5')

<Figure size 1304x304 with 5 Axes>

In [21]:
df_marker = pd.read_csv('./210430_clusters/markergenes_manual.csv', index_col=0)
df_marker = df_marker.loc[list_label]

dict_markers = {pos:[row[0], row[1], row[2]] for pos,row in df_marker.iterrows()}

In [22]:
sc.pl.dotplot(adata, dict_markers, 'Bcell_cluster', categories_order=list_label, dendrogram=False, swap_axes=True, 
              save='minor_manual_3_vert')

<Figure size 356.8x752 with 5 Axes>

## plasma

In [3]:
adata = sc.read(results_file_Bcell)

In [4]:
list_label = ['Naive B cell', 
              'Pre GC B cell', 'GC B cell', 'Thymic memory B cell', 'Unswitched memory B cell',
              'Memory B cell (I)', 'Memory B cell (II)', 'Plasmablast']

In [12]:
sc.pl.dotplot(adata, ['CD19', 'MS4A1', 'CD38', 'SDC1', 'CXCR4', 'PRDM1'], 
              'Bcell_cluster', categories_order=list_label, dendrogram=False, save=False)

<Figure size 297.6x304 with 4 Axes>

In [43]:
sc.pl.dotplot(adata, ['CD19', 'MS4A1', 'CD38', 'SDC1', 'CXCR4', 'PAX5', 'IRF4', 'XBP1', 'PRDM1'], 
              'Bcell_cluster', categories_order=list_label, dendrogram=False, save=False)

<Figure size 386.4x304 with 4 Axes>

In [44]:
sc.pl.umap(adata, color=['CD19', 'MS4A1', 'CD38', 'SDC1', 'CXCR4', 'PAX5', 'IRF4', 'XBP1', 'PRDM1'])

<Figure size 1545.6x960 with 18 Axes>

In [14]:
sc.pl.umap(adata, color='Bcell_cluster', groups=['Plasmablast'])

<Figure size 320x320 with 1 Axes>

## Thymus vs Peripheral

In [25]:
adata = sc.read(results_file_Bcell)

In [27]:
list_clu = ['GC B cell', 'Memory B cell (I)', 'Memory B cell (II)', 'Naive B cell', 'Plasmablast', 
            'Pre GC B cell', 'Thymic memory B cell', 'Unswitched memory B cell']

In [28]:
list_thymic_genes = []
for clu in list_clu:
    a = adata[adata.obs['Bcell_cluster'] == clu]
    sc.tl.rank_genes_groups(a, 'site')
#     sc.pl.rank_genes_groups(a)
    result = a.uns['rank_genes_groups']
    groups = result['names'].dtype.names
    df_res = pd.DataFrame(
        {group + '_' + key[:1]: result[key][group]
        for group in groups for key in ['names', 'pvals_adj', 'logfoldchanges']})
    list_thymic_genes.append(list(df_res.loc[(df_res['Thymus_p'] < 0.05) & (df_res['Thymus_l'] > 1), 'Thymus_n']))

ranking genes
Trying to set attribute `.uns` of view, copying.
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)
ranking genes
Trying to set attribute `.uns` of view, copying.
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)
ranking genes
Trying to set attribute `.uns` of view, copying.
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted 

In [29]:
import collections

In [30]:
c = collections.Counter([item for sublist in list_thymic_genes for item in sublist])

In [38]:
plt.figure(figsize=(3,12))
pd.Series(c).sort_values(ascending=False).head(40).sort_values().plot.barh()
plt.xlabel('UP in thymus')
plt.grid(None)
plt.savefig(str(sc.settings.figdir) + '/barthymusperi.pdf', bbox_inches='tight')

<Figure size 240x960 with 1 Axes>

In [39]:
adata.obs['cluster:site'] = adata.obs['Bcell_cluster'].astype(str) + ':' + adata.obs['site'].astype(str).str[0]

In [ ]:
sc.pl.dotplot(adata[adata.obs['CD4T_cluster'].isin(list_clu)],
              pd.Series(c).sort_values(ascending=False).head(10).index, groupby='cluster:site',
             swap_axes=True, save='thymus_peri')

## Gene set analysis

In [123]:
adata = sc.read(results_file_Bcell)

In [124]:
sc.tl.rank_genes_groups(adata, 'Bcell_cluster', method='t-test_overestim_var')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:02)


<Figure size 1280x640 with 8 Axes>

In [125]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).to_csv('scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gene.rank')

In [126]:
import gseapy as gp
from gseapy.plot import barplot, dotplot
gp.__version__

import os

In [127]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
df_res = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'logfoldchanges', 'pvals_adj', 'scores']})

In [ ]:
for clu in adata.obs['Bcell_cluster'].unique():
    os.makedirs('scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_reactome/gseapy/{}'.format(clu), exist_ok=True)
    df_res[['{}_n'.format(clu), '{}_s'.format(clu)]].to_csv('scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_reactome/{}.rnk'.format(clu), 
                                                            sep='\t', header=None, index=None)
    pre_res = gp.prerank(rnk='scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_reactome/{}.rnk'.format(clu), 
                         gene_sets='data/ReactomePathways.gmt',
                     processes=10,
                     permutation_num=1000, # reduce number to speed up testing
                     outdir='scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_reactome/gseapy/{}'.format(clu), 
                         format='png', seed=6)

In [ ]:
for clu in adata.obs['Bcell_cluster'].unique():
    os.makedirs('scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_wp/gseapy/{}'.format(clu), exist_ok=True)
    df_res[['{}_n'.format(clu), '{}_s'.format(clu)]].to_csv('scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_wp/{}.rnk'.format(clu), 
                                                            sep='\t', header=None, index=None)
    pre_res = gp.prerank(rnk='scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_reactome/{}.rnk'.format(clu), 
                         gene_sets='WikiPathways_2019_Human',
                     processes=10,
                     permutation_num=1000, # reduce number to speed up testing
                     outdir='scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_wp/gseapy/{}'.format(clu), 
                         format='png', seed=6)

In [ ]:
for clu in adata.obs['Bcell_cluster'].unique():
    os.makedirs('scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_kegg/gseapy/{}'.format(clu), exist_ok=True)
    df_res[['{}_n'.format(clu), '{}_s'.format(clu)]].to_csv('scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_kegg/{}.rnk'.format(clu), 
                                                            sep='\t', header=None, index=None)
    pre_res = gp.prerank(rnk='scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_reactome/{}.rnk'.format(clu), 
                         gene_sets='KEGG_2021_Human',
                     processes=10,
                     permutation_num=1000, # reduce number to speed up testing
                     outdir='scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_kegg/gseapy/{}'.format(clu), 
                         format='png', seed=6)

In [ ]:
for clu in adata.obs['Bcell_cluster'].unique():
    os.makedirs('scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_gobp/gseapy/{}'.format(clu), exist_ok=True)
    df_res[['{}_n'.format(clu), '{}_s'.format(clu)]].to_csv('scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_gobp/{}.rnk'.format(clu), 
                                                            sep='\t', header=None, index=None)
    pre_res = gp.prerank(rnk='scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_reactome/{}.rnk'.format(clu), 
                         gene_sets='GO_Biological_Process_2018',
                     processes=10,
                     permutation_num=1000, # reduce number to speed up testing
                     outdir='scanpy/210711_merged_thymoma_MG21.22.23.03_Bcell/gsea_gobp/gseapy/{}'.format(clu), 
                         format='png', seed=6)

In [ ]:
with open('./data/ReactomePathways.gmt', 'r') as f:
    txt = f.read()
    
df_reactome = pd.DataFrame([[x.split('\t')[0], x.split('\t')[1], x.split('\t')[2:]] for x in txt.split('\n') if len(x.split('\t')) > 10],
            columns=['geneset', 'id', 'genes'])
df_reactome.index = df_reactome['id']
df_reactome.head()

In [ ]:
id_gs = 'R-HSA-2132295'

sc.tl.score_genes(adata, df_reactome.loc[id_gs, 'genes'], score_name='score_{}'.format(id_gs))
sc.pl.violin(adata, ['score_{}'.format(id_gs)], groupby='Bcell_cluster', log=False,
             rotation=90, save='reactome/{}.pdf'.format(id_gs))

In [ ]:
sc.pl.dotplot(adata, ['HLA-DQB1', 'HLA-DRB1', 'STAT1', 'IRF1', 'CIITA'], groupby='Bcell_cluster', title='HLA class II signaling',
             save='hlaclassII.pdf')

## Velocyto

In [152]:
import scvelo as scv
mpl.rcParams['axes.grid'] = False

In [153]:
adata = sc.read('./data/velocyto.210430.merged.loom')
adata.var_names_make_unique()
adata.obs.index = [x.split(':')[1][:-1] for x in adata.obs.index]
adata.obs_names_make_unique()

Only considering the two last: ['.merged', '.loom'].
Only considering the two last: ['.merged', '.loom'].
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [154]:
mdata = sc.read(results_file_Bcell)

In [155]:
mdata = mdata[mdata.obs['site'] == 'Thymus']
list_overlap_cells = list(set(adata.obs.index) & set(mdata.obs.index))
adata = adata[list_overlap_cells,:]
mdata = mdata[list_overlap_cells,:]
adata.obs['Bcell_cluster'] = mdata.obs['Bcell_cluster']
adata.uns['Bcell_cluster_colors'] = mdata.uns['Bcell_cluster_colors']
adata.obsm['X_umap'] = mdata.obsm['X_umap']

Trying to set attribute `.obs` of view, copying.


In [156]:
scv.pl.proportions(adata, groupby="Bcell_cluster")
# plt.grid(False)

<Figure size 1000x200 with 2 Axes>

In [157]:
scv.pl.proportions(adata, groupby="sample")
# plt.grid(False)

<Figure size 1000x200 with 1 Axes>

In [158]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

computing PCA
    on highly variable genes
    with n_comps=30


Filtered out 32491 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.


    finished (0:00:00)


computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [159]:
scv.tl.recover_dynamics(adata)

recovering dynamics (using 1/16 cores)


  0%|          | 0/587 [00:00<?, ?gene/s]

    finished (0:01:08) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)


In [160]:
scv.tl.velocity(adata)#, mode='stochastic', filter_genes=True)
scv.tl.velocity_graph(adata)#, n_recurse_neighbors=5)

computing velocities
    finished (0:00:00) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph
    finished (0:00:00) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [161]:
scv.tl.velocity_embedding(adata, basis='umap', direct_pca_projection=False)
# scv.tl.velocity_embedding(adata, basis='draw_graph_fr')

computing velocity embedding
    finished (0:00:00) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)


In [162]:
scv.pl.velocity_embedding(adata, basis='umap', color='Bcell_cluster', 
                          legend_loc='on data', scale=.5, figsize=(10,10))
# scv.pl.velocity_embedding(adata, basis='draw_graph_fr', color='louvain', scale=3, legend_loc='on data')

<Figure size 800x800 with 1 Axes>

In [163]:
scv.pl.velocity_embedding_grid(adata, basis='umap', color='Bcell_cluster', 
                          legend_loc='on data', scale=0.2, figsize=(5,5))

<Figure size 400x400 with 1 Axes>

In [164]:
scv.pl.velocity_embedding_stream(adata, basis='umap', color='Bcell_cluster', 
                                 legend_loc='right mergin', figsize=(5,5), save='thymus_bcell_stream.pdf')

figure cannot be saved as pdf, using png instead.
saving figure to file ./figures/scvelo_thymus_bcell_stream.png


<Figure size 400x400 with 1 Axes>

In [165]:
scv.tl.terminal_states(adata)
scv.pl.scatter(adata, color=['root_cells', 'end_points'])

computing terminal states
    identified 3 regions of root cells and 1 region of end points .
    finished (0:00:00) --> added
    'root_cells', root cells of Markov diffusion process (adata.obs)
    'end_points', end points of Markov diffusion process (adata.obs)


<Figure size 640x320 with 4 Axes>

In [166]:
scv.tl.velocity_confidence(adata)
scv.pl.scatter(adata, color='velocity_confidence', perc=[2,98])

--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)


<Figure size 320x320 with 2 Axes>

In [167]:
scv.tl.latent_time(adata, root_key="Naive B cell")
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80)

computing latent time using root_cells as prior
    finished (0:00:00) --> added 
    'latent_time', shared time (adata.obs)


<Figure size 320x320 with 2 Axes>

In [168]:
scv.tl.rank_velocity_genes(adata, groupby='Bcell_cluster', min_corr=.3)

df = scv.DataFrame(adata.uns['rank_velocity_genes']['names'])
df.head()

ranking velocity genes
    finished (0:00:00) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)


,GC B cell,Memory B cell (I),Memory B cell (II),Naive B cell,Plasmablast,Pre GC B cell,Thymic memory B cell,Unswitched memory B cell
0,AFF2,TBXAS1,SYT1,GAB1,SPCS2,DEK,PIP4K2A,CD83
1,AC104170.1,IFT57,PIP4K2A,ZBTB16,IGHGP,SYNE2,TBXAS1,IFI44L
2,NUGGC,PARP14,ARL6IP5,FOXP1,PTMA,BCL7A,SYT1,ARL6IP5
3,PAG1,SYT1,TOX,ZNF318,HMCES,SMC4,HIPK2,SERPINB9
4,NEDD4L,CD86,ADAM28,COL19A1,LIME1,SMC1A,BLK,REL


In [169]:
scv.pl.velocity(adata, var_names=df.head(5)['GC B cell'],
                color='Bcell_cluster', basis='umap', figsize=(10,10))

<Figure size 1200x2000 with 25 Axes>

In [170]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:300]
scv.pl.heatmap(adata, var_names=top_genes, sortby='latent_time', 
               col_color='Bcell_cluster', 
               n_convolve=100)

<Figure size 640x320 with 4 Axes>

## BCR

In [171]:
import scirpy as ir

In [172]:
adata = sc.read(results_file_Bcell_minor_cluster)
adata = adata[adata.obs['sample'].isin(['MG03_PB', 'MG03_TL'])]

In [173]:
df_ir = pd.concat([pd.read_csv('data/cellranger/MG03_PB_BCR/filtered_contig_annotations.csv'),
pd.read_csv('data/cellranger/MG03_TL_BCR/filtered_contig_annotations.csv')])
df_ir['barcode'] = df_ir['barcode'].str.split('-').str.get(0)
df_ir.to_csv('data/cellranger/MG03_BCR.csv', index=None)

In [174]:
mg03_ir = ir.io.read_10x_vdj('data/cellranger/MG03_BCR.csv')
ir.pp.merge_with_ir(adata, mg03_ir)

In [175]:
adata[adata.obs['has_ir'] == 'True'].obs['sample'].value_counts()

MG03_PB    8186
MG03_TL       9
Name: sample, dtype: int64

In [176]:
mg03_ir.obs

,multi_chain,extra_chains,is_cell,high_confidence,IR_VJ_1_c_call,IR_VJ_2_c_call,IR_VDJ_1_c_call,IR_VDJ_2_c_call,IR_VJ_1_consensus_count,IR_VJ_2_consensus_count,...,IR_VDJ_2_sequence_id,IR_VJ_1_v_call,IR_VJ_2_v_call,IR_VDJ_1_v_call,IR_VDJ_2_v_call,IR_VJ_1_v_cigar,IR_VJ_2_v_cigar,IR_VDJ_1_v_cigar,IR_VDJ_2_v_cigar,has_ir
obs_names,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGACACTAA,False,[],True,True,IGLC2,NaN,IGHM,NaN,4867.0,NaN,...,NaN,IGLV1-40,NaN,IGHV4-30-2,NaN,NaN,NaN,NaN,NaN,True
AAACCTGAGCGATAGC,False,[],True,True,IGLC3,NaN,IGHM,NaN,1660.0,NaN,...,NaN,IGLV3-25,NaN,IGHV3-30,NaN,NaN,NaN,NaN,NaN,True
AAACCTGAGCGCTCCA,False,[],True,True,IGKC,NaN,IGHA1,NaN,7328.0,NaN,...,NaN,IGKV1D-39,NaN,IGHV3-7,NaN,NaN,NaN,NaN,NaN,True
AAACCTGAGGCAGTCA,False,[],True,True,IGKC,NaN,IGHM,NaN,7198.0,NaN,...,NaN,IGKV3-20,NaN,IGHV4-61,NaN,NaN,NaN,NaN,NaN,True
AAACCTGAGTGCAAGC,False,[],True,True,IGLC3,NaN,IGHG3,NaN,4793.0,NaN,...,NaN,IGLV3-21,NaN,IGHV3-33,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCATGCTCC,False,[],True,True,IGKC,NaN,IGHM,NaN,7672.0,NaN,...,NaN,IGKV3-11,NaN,IGHV4-34,NaN,NaN,NaN,NaN,NaN,True
TTTGTCATCGAGAACG,False,[],True,True,IGKC,IGKC,IGHM,IGHM,11569.0,4657.0,...,NaN,IGKV3-11,IGKV1-12,IGHV4-34,IGHV1-46,NaN,NaN,NaN,NaN,True
TTTGTCATCGTCTGCT,False,[],True,True,IGKC,NaN,IGHA1,NaN,8443.0,NaN,...,NaN,IGKV1D-39,NaN,IGHV3-23,NaN,NaN,NaN,NaN,NaN,True


In [177]:
adata[adata.obs['has_ir'] == 'True'].obs['minor_cluster'].value_counts()

Naive B cell                4589
Unswitched memory B cell    1347
Memory B cell (I)            927
Memory B cell (II)           465
Pre GC B cell                286
Plasmablast                  244
Thymic memory B cell         182
GC B cell                    151
Doublet                        4
Name: minor_cluster, dtype: int64

In [178]:
ir.tl.chain_qc(adata)

In [179]:
ax = ir.pl.group_abundance(adata, groupby="receptor_subtype", target_col="sample")

... storing 'receptor_type' as categorical
... storing 'receptor_subtype' as categorical
... storing 'chain_pairing' as categorical


<Figure size 412.8x309.6 with 1 Axes>

In [180]:
ir.pp.ir_dist(
    adata,
    metric="alignment",
    sequence="aa",
    cutoff=15,
)

Computing sequence x sequence distance matrix for VJ sequences.


  0%|          | 0/2850 [00:00<?, ?it/s]

Computing sequence x sequence distance matrix for VDJ sequences.


  0%|          | 0/14028 [00:00<?, ?it/s]

In [181]:
ir.tl.define_clonotype_clusters(
    adata, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any"
)

Initializing lookup tables. 
--> Done initializing lookup tables. (0:00:00)
Computing clonotype x clonotype distances.
NB: Computation happens in chunks. The progressbar only advances when a chunk has finished. 


  0%|          | 0/8068 [00:00<?, ?it/s]

--> Done computing clonotype x clonotype distances.  (0:00:26)
Stored clonal assignments in `adata.obs["cc_aa_alignment"]`.


In [182]:
ir.tl.clonotype_network(adata, min_cells=3, sequence="aa", metric="alignment")

In [183]:
ir.pl.clonotype_network(
    adata, color="minor_cluster", label_fontsize=9, panel_size=(7, 7), base_size=20
)

... storing 'cc_aa_alignment' as categorical


<AxesSubplot:>

<Figure size 760x560 with 4 Axes>

In [184]:
ir.tl.define_clonotypes(adata)
ir.tl.clonal_expansion(adata)
sc.pl.umap(adata, color=["clonal_expansion"])

ir_dist for sequence='nt' and metric='identity' not found. Computing with default parameters.
Computing sequence x sequence distance matrix for VJ sequences.
Computing sequence x sequence distance matrix for VDJ sequences.
Initializing lookup tables. 
--> Done initializing lookup tables. (0:00:00)
Computing clonotype x clonotype distances.
NB: Computation happens in chunks. The progressbar only advances when a chunk has finished. 


  0%|          | 0/8075 [00:00<?, ?it/s]

--> Done computing clonotype x clonotype distances.  (0:00:09)
Stored clonal assignments in `adata.obs["clone_id"]`.
... storing 'clone_id' as categorical
... storing 'clonal_expansion' as categorical


<Figure size 320x320 with 1 Axes>

In [185]:
ir.pl.clonal_expansion(adata, "minor_cluster",figsize=(10,3))

<AxesSubplot:>

<Figure size 1200x360 with 1 Axes>

In [186]:
adata = adata[~adata.obs['clone_id'].isna()]
ir.tl.repertoire_overlap(adata, "minor_cluster", inplace=True)
ir.pl.repertoire_overlap(adata, "minor_cluster")

Trying to set attribute `.uns` of view, copying.


<Figure size 800x800 with 4 Axes>